# 🎨 모델 맞춤화 소개

## 🛒 Zava 시나리오

Cora는 기본 모델로 잘 작동하고 있지만, Zava는 자사 브랜드의 어조, 주택 개량 관련 용어, 고객 서비스 지침에 더욱 부합하는 응답을 제공하기를 원합니다. 일반적인 모델은 주택 개량 소매업의 특수한 뉘앙스를 제대로 반영하지 못할 수 있습니다.

**기회**: 단순히 즉각적인 엔지니어링에만 의존하는 대신, 미세 조정 및 정제와 같은 기법을 통해 Cora 모델을 Zava의 주택 개량 소매업 요구사항에 더욱 특화시켜 맞춤화할 수 있습니다.

## 학습 목표

이 섹션에서는 다음 내용을 이해하게 됩니다.

1. **소규모 프롬프트(Few-shot prompting)** - 프롬프트의 예시를 통해 모델을 학습시키는 방법
2. **지도 미세 조정(Supervised Fine-Tuning, SFT)** - 도메인별 데이터를 사용하여 모델을 학습시키는 방법
3. **데이터 정제(Distillation)** - 규모가 큰 모델에서 작은 모델로 지식을 이전하는 방법
4. **미세 조정을 위한 데이터 준비** - JSONL 형식 및 모범 사례
5. **각 사용자 지정 기법의 사용 시점** - 장단점 및 사용 사례

## 왜 이것이 중요한가?

모델 사용자 지정을 통해 다음과 같은 이점을 얻을 수 있습니다.
- **도메인별 작업에 대한 응답 품질 향상**
- **더 작고 특화된 모델 사용**으로 **비용 절감**
- **브랜드 보이스 및 가이드라인에 맞춘 출력**
- **비즈니스 고유의 전문 용어 처리**

Zava의 주택 개량 소매 사업에 Cora를 더욱 효과적으로 활용하기 위해 모델을 사용자 지정하는 방법을 살펴보겠습니다.

---

## 모델을 맞춤 설정하는 이유

기본 모델(GPT-4o, GPT-4o-mini)은 일반 인터넷 데이터를 기반으로 학습됩니다. 강력하지만 범용성이 떨어집니다.

**기본 모델의 문제점:**

### 1. 일관성 없는 어조/스타일

**고객:** "어떤 페인트가 있나요?"

**기본 모델:** "저희는 실내용, 실외용, 라텍스, 유성 등 다양한 페인트 옵션을 보유하고 있습니다..."

**맞춤 모델(Zava 스타일):** "좋은 질문입니다! 고객님의 프로젝트에 적합한 훌륭한 페인트 옵션이 여러 가지 있습니다. 완벽한 색상을 찾도록 도와드리겠습니다..."

맞춤 모델은 Zava의 친절하고 도움이 되는 브랜드 어조를 일관되게 유지합니다.

### 2. 전문 지식 부족

**고객:** "T1-11 사이딩에 사용할 페인트가 필요합니다."

**기본 모델:** "페인트를 찾는 데 도움을 드릴 수 있습니다. 어떤 색상을 찾으시나요?"

**맞춤 모델:** "T1-11 사이딩에는 침투력이 좋은 고품질 외장용 아크릴 라텍스 페인트가 필요합니다. 저희 프리미엄 외장 페인트(PFIP000002)가 이 용도에 딱 맞습니다..."

맞춤 모델은 하드웨어/건축 용어를 이해합니다.

### 3. 긴 프롬프트

**맞춤 설정이 없는 경우:** 모든 프롬프트에 예시를 포함해야 합니다.
```
프롬프트(500 토큰):
"당신은 친절한 Zava 도우미 Cora입니다. 예시:
질문: 어떤 페인트를 사용하시나요? 답변: [예시]
질문: 어떤 드릴을 사용하시나요? 답변: [예시]
...
이제 다음 질문에 답해 주세요: 어떤 페인트를 가지고 계신가요?"
``

**맞춤 설정이 있는 경우:** 모델이 이미 스타일을 "알고" 있습니다.
```
프롬프트(50 토큰):
"당신은 Cora입니다. 고객이 묻습니다: 어떤 페인트를 가지고 계신가요?"
```

**결과:** 토큰 수의 상당한 감소로 대규모 운영 시 비용 절감

### 4. 응답 일관성

**기본 모델 응답의 다양성:**
- 질의 1: 공식적이고 기술적인 내용
- 질의 2: 비공식적이고 간략한 내용
- 질의 3: 지나치게 상세한 내용

**맞춤형 모델:** 모든 응답에서 일관된 어조, 구조 및 상세 수준 유지

## 맞춤 설정 접근 방식

모델 동작을 맞춤 설정하는 주요 방법은 세 가지입니다.

| 접근 방식 | 기능 | 최적 사용 분야 | 비용 | 노력 |
----------|--------------|----------|------|--------|
**소규모 질의 응답(Few-Shot Prompting)** | 질의 응답에 예시 포함 | 빠른 테스트, 동적 예시 | 낮은 비용(토큰당 지불) | 낮은 비용 |
**미세 조정** | 자체 데이터로 모델 학습 | 일관된 스타일, 도메인 지식 | 중간 비용(일회성 학습) | 중간 비용 |
**핵심 정보 추출** | 대규모 모델의 지식 이전 | 비용 최적화, 효율성 | 낮음-중간 | 높음 |

각 항목을 자세히 살펴보겠습니다.

---

## Few-Shot Prompting

**Few-shot prompting**이란 모델의 동작을 안내하기 위해 프롬프트에 예시 쿼리-응답 쌍을 포함하는 것을 의미합니다.

### How It Works

```python
prompt = """
You are Cora, a helpful Zava Hardware assistant.

Example 1:
Customer: What paint do you have?
Cora: Great question! We have several excellent paint options. For interior 
projects, I recommend our Premium Interior Paint. For exterior, our Premium 
Exterior Paint is weather-resistant and durable. What's your project?

Example 2:
Customer: Is PFIP000002 in stock?
Cora: Yes! Premium Exterior Paint (SKU: PFIP000002) is currently in stock 
with 75 units available. Would you like me to help you with anything else?

Now answer:
Customer: {user_query}
Cora:
"""
```

**이 모델은 예시를 통해 학습하고 스타일을 모방합니다.

### Pros

✅ **빠른 구현** - 교육 불필요
✅ **유연성** - 언제든지 예시 변경 가능
✅ **인프라 불필요** - 프롬프트만 수정하면 됨
✅ **동적** - 시나리오별로 다른 예시 제공

### 단점

❌ **토큰 비용 증가** - 예시 추가 시 요청당 200~500 토큰 추가
❌ **컨텍스트 윈도우 사용** - 실제 대화 공간 부족
❌ **일관성 부족** - 모델이 여전히 즉흥적으로 작동함
❌ **제한된 예시** - 5~10개 예시만 가능

### Few-Shot을 언제 사용하는가

- ✅ 프로토타이핑 및 테스트
- ✅ 유연성 필요 (예시가 자주 변경됨)
- ✅ 낮은 쿼리량 (< 하루 1,000건)
- ✅ 간단한 동작 변경
- ❌ 높은 쿼리량 (비용 부담)
- ❌ 완벽한 일관성 필요
- ❌ 복잡한 도메인 지식 필요

### Example Calculation

**Scenario:** 10,000 queries/day with 5 examples (300 tokens)

```
소량 데이터 처리 방식:
- 요청당 토큰 사용량 증가(예제 포함)
- 사용량 기반 요금제
- 비용은 처리량에 비례하여 증가

정밀 조정 방식:
- 요청당 토큰 사용량 감소(예제 불필요)
- 일회성 학습 비용
- 요청당 추론 비용 감소

처리량이 많을수록 정밀 조정 방식이 더 비용 효율적입니다.

**결론:** 소량 데이터 처리에는 소량 데이터 처리 방식이 적합하고, 대규모 처리에는 정밀 조정 방식이 더 효과적입니다.

For current pricing: https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/
```

---

## Fine-Tuning

**Fine-tuning**은 사용자의 특정 데이터를 기반으로 기본 모델을 지속적으로 학습시키고, 내부 매개변수를 조정하여 사용자의 도메인과 스타일을 학습시키는 것을 의미합니다.

### How It Works

```
Base Model (GPT-4o-mini)
         ↓
    + Your Training Data
      (100-1000 examples)
         ↓
   Fine-Tuning Process
   (2-6 hours on Azure)
         ↓
   Custom Fine-Tuned Model
   (your-model-deployment)
```

**모델의 가중치가 업데이트**되어 패턴을 직접 인코딩합니다.

### What Gets Trained

**Training data format (JSONL):**

```jsonl
{"messages": [{"role": "system", "content": "You are Cora, a Zava assistant"}, {"role": "user", "content": "What paint do you have?"}, {"role": "assistant", "content": "Great question! We have several excellent paint options..."}]}
{"messages": [{"role": "system", "content": "You are Cora, a Zava assistant"}, {"role": "user", "content": "Is PFIP000002 in stock?"}, {"role": "assistant", "content": "Yes! Premium Exterior Paint is currently in stock..."}]}
```

각 줄은 다음과 같은 내용을 보여주는 완전한 대화 예시입니다.
1. 시스템 지시사항
2. 사용자 질문
3. 예상되는 어시스턴트 응답

**모델은 다음을 학습합니다.**
- 응답 스타일 및 어조
- 도메인 용어
- 응답 구조 및 형식
- 브랜드 보이스 및 개성

### Supervised Fine-Tuning (SFT)

**SFT**는 가장 일반적인 미세 조정 접근 방식입니다.

1. **지도 학습** - 레이블이 지정된 예제(질문 → 예상 응답)를 제공합니다.
2. **학습 목표** - 모델은 입력값을 기반으로 응답을 예측하도록 학습합니다.
3. **경사 하강법** - 모델 가중치를 조정하여 데이터에 대한 오류를 최소화합니다.

**비유:** 연습 문제와 정답을 통해 학생을 지도하는 것과 같습니다.

### 학습 데이터 준비

**데이터 요구 사항:**
- **최소:** 50개 예제 (많을수록 좋음)
- **권장:** 100~1000개 예제
- **형식:** JSONL (JSON 라인)
- **품질:** 고품질 예제만 사용

**데이터 소스:**
1. **실제 대화** (가능한 경우)
2. **합성 생성** (Azure AI 시뮬레이터 사용)
3. **수동 큐레이션** (전문가가 작성한 예제)
4. **하이브리드** (실제 + 증강)

**Example generation:**

```python
training_data = [
    {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "What paint is best for kitchens?"},
            {"role": "assistant", "content": "For kitchens, I recommend our Premium Interior Paint..."}
        ]
    },
    # ... 99 more examples
]

# Save as JSONL
with open("training.jsonl", "w") as f:
    for item in training_data:
        f.write(json.dumps(item) + "\n")
```

### Token Optimization

**Problem:** 토큰 개수에 따른 학습 비용

**Formula:**
```
Training Cost = (Total Tokens in Dataset) × (Training Epochs) × (Price per Token)
```

**Optimization strategies:**

1. **Remove redundancy**
```python
# Before (verbose)
"Hello! Thank you for asking! I'm happy to help you with that question..."

# After (concise)
"I recommend our Premium Exterior Paint for outdoor wood projects..."
```

2. **Truncate long responses**
```python
# Keep responses under 150 tokens
if token_count(response) > 150:
    response = truncate_intelligently(response, 150)
```

3. **Validate before uploading**
```python
from azure.ai.ml import MLClient

# Check token counts
total_tokens = sum(count_tokens(ex) for ex in training_data)
cost_estimate = (total_tokens * epochs * price_per_1k) / 1000

print(f"Estimated training cost: ${cost_estimate:.2f}")
```

### Fine-Tuning Process

**Steps:**

1. **Prepare data** (JSONL format)
2. **Upload to Azure OpenAI**
3. **Submit fine-tuning job**
4. **Monitor progress** (2-6 hours)
5. **Deploy fine-tuned model**
6. **Test and validate**

**Azure OpenAI fine-tuning job:**

```python
from openai import AzureOpenAI

client = AzureOpenAI(...)

# Upload training file
with open("training.jsonl", "rb") as f:
    training_file = client.files.create(file=f, purpose="fine-tune")

# Create fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-4o-mini-2024-07-18",  # Base model
    hyperparameters={
        "n_epochs": 3  # Number of training passes
    }
)

# Monitor
status = client.fine_tuning.jobs.retrieve(job.id)
print(f"Status: {status.status}")
```

### 장점

✅ **짧은 프롬프트** - 예시 불필요
✅ **일관된 동작** - 모델이 사용자의 스타일을 파악
✅ **향상된 도메인 지식** - 전문 용어 학습
✅ **대규모 환경에서 비용 효율성** - 쿼리당 비용 절감
✅ **향상된 품질** - 특정 사용 사례에 특화

### 단점

❌ **초기 노력** - 학습 데이터 생성 필요
❌ **학습 시간** - 작업당 2~6시간
❌ **정적 지식** - 업데이트 시 재학습 필요
❌ **복잡한 버전 관리** - 모델 버전 관리 필요

### 미세 조정 사용 시점

- ✅ 높은 쿼리량 (하루 1,000건 이상)
- ✅ 일관된 어조/스타일 필요
- ✅ 도메인별 전문 용어 필요
- ✅ 고품질 학습 데이터 보유
- ✅ 장기 배포
- ❌ 데이터가 매일 변경되는 경우
- ❌ 실시간 업데이트가 필요합니다
- ❌ 조회량이 매우 적음

---

## Distillation

**Distillation**은 더 작고 빠른 모델을 훈련시켜 더 크고 성능이 뛰어난 모델을 모방하는 것을 의미합니다.

### How It Works

```
Large "Teacher" Model          Small "Student" Model
(Higher cost/better)    →      (Lower cost/faster)
                                     
Query: "What paint?"    →      Query: "What paint?"
Response: [detailed]    →      Response: [similar quality]
                                     
Cost: Higher per token         Cost: Lower per token
Latency: Slower                Latency: Faster

See pricing: https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/
```

**Goal:** GPT-4o 수준의 품질을 GPT-4o-mini 수준의 비용/속도로 누리세요.

### Distillation Process

**Step 1: Generate Teacher Responses**

```python
# Use large model to generate high-quality responses
teacher_model = "gpt-4o"
student_training_data = []

for query in training_queries:
    teacher_response = call_model(teacher_model, query)
    student_training_data.append({
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
            {"role": "assistant", "content": teacher_response}
        ]
    })
```

**Step 2: Fine-Tune Student Model**

```python
# Fine-tune small model on teacher's responses
student_model = fine_tune(
    base_model="gpt-4o-mini",
    training_data=student_training_data
)
```

**Step 3: Evaluate**

```python
# Compare student vs teacher
for query in test_queries:
    teacher_response = call_model("gpt-4o", query)
    student_response = call_model(student_model, query)
    
    similarity = compute_similarity(teacher_response, student_response)
    print(f"Similarity: {similarity}")  # Goal: > 0.85
```

### Knowledge Transfer

**What gets distilled:**
- Reasoning patterns
- Response structure
- Domain knowledge
- Task-specific behaviors

**What doesn't get distilled:**
- Raw intelligence (student has limits)
- Emergent capabilities (student is smaller)
- Perfect accuracy (some quality loss acceptable)

### Types of Distillation

**1. Basic Distillation**
- Student learns from teacher's outputs directly
- Simple, effective for most use cases

**2. Distillation with Custom Graders**
- Use custom evaluators to score teacher responses
- Only keep high-quality examples for student training
- Better quality control

**Example: Custom grader**

```python
def grade_response(query, response):
    """Custom evaluator for response quality"""
    score = 0
    
    # Check for key elements
    if contains_product_sku(response):
        score += 1
    if polite_tone(response):
        score += 1
    if factually_grounded(response):
        score += 1
    if under_token_limit(response, 150):
        score += 1
        
    return score >= 3  # Keep if passes quality threshold

# Filter training data
high_quality_data = [
    ex for ex in distillation_data 
    if grade_response(ex["query"], ex["response"])
]
```

### 장점

✅ **비용 절감** - 비슷한 품질의 더 저렴한 모델
✅ **속도 향상** - 더 빠른 추론
✅ **소규모 배포** - 더 적은 리소스 요구 사항
✅ **품질 유지** - 교사 모델의 품질을 약 85~95% 유지

### 단점

❌ **2단계 프로세스** - 생성 + 미세 조정
❌ **품질 한계** - 학생 모델의 용량을 초과할 수 없음
❌ **초기 비용** - 교사 응답 생성 비용이 높음
❌ **복잡한 평가** - 품질 유지 검증 필요

### Distillation을 언제 사용하는가

- ✅ 프로덕션 환경에서 비용이 많이 드는 대형 모델을 사용하는 경우
- ✅ 비용/지연 시간을 줄여야 하는 경우
- ✅ 교사 모델 생성 예산이 있는 경우
- ✅ 5~15%의 품질 저하를 허용할 수 있는 경우
- ❌ 이미 가장 작은 모델을 사용 중인 경우
- ❌ 최고 품질이 필요한 경우 (타협할 수 없음)
- ❌ 쿼리량이 적은 경우 (ROI가 높은 경우) (너무 낮음)

### ROI 계산 예시

**시나리오:** 월 10만 건의 쿼리
기존 방식(대형 모델):
- 토큰당 비용 높음
- 월별 운영 비용 높음

최적화 방식(소형 모델, 미세 조정):
- 일회성 생성 비용(교사 응답)
- 일회성 학습 비용
- 토큰당 추론 비용 감소
- 월별 운영 비용 대폭 절감

Result: Substantial cost savings at high volume

Typical payback period: Days to weeks

Annual savings: $15,000

For pricing details: https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/

```
**Conclusion:** Distillation is extremely valuable at high volume.

---

## Choosing the Right Approach

### Decision Tree

```
Start: Need to customize model behavior?
  ↓
  Yes → High query volume (> 1000/day)?
         ↓
         Yes → Using expensive model?
                ↓
                Yes → Use DISTILLATION
                       (GPT-4o → GPT-4o-mini fine-tuned)
                ↓
                No → Use FINE-TUNING
                      (GPT-4o-mini base → fine-tuned)
         ↓
         No → Need flexibility?
               ↓
               Yes → Use FEW-SHOT PROMPTING
               ↓
               No → Use FINE-TUNING
                     (better long-term)
```

### Comparison Matrix

| Factor | Few-Shot | Fine-Tuning | Distillation |
|--------|----------|-------------|--------------|
| **Setup Time** | Minutes | Hours | Days |
| **Query Volume Sweet Spot** | < 1K/day | > 1K/day | > 10K/day |
| **Consistency** | Medium | High | High |
| **Cost at 10K queries/day** | Higher | Medium | Lower |
| **Flexibility** | High | Low | Low |
| **Domain Knowledge** | Limited | Good | Good |
| **Quality** | Good | Better | Best (if done right) |

### Hybrid Approaches

**Combine multiple techniques:**

**1. Fine-Tuning + RAG**
- Fine-tune for tone/style (static)
- RAG for product knowledge (dynamic)

```python
# Fine-tuned model for Zava brand voice
model = "zava-custom-model"

# RAG for up-to-date product info
context = retrieve_from_search(query)

# Combine
response = model.query(
    prompt=f"Context: {context}\n\nQuestion: {query}",
    model=model
)
```

**2. Few-Shot + Fine-Tuning**
- Fine-tune for general behavior
- Few-shot for specific edge cases

```python
# Use fine-tuned model as base
model = "zava-fine-tuned"

# Add few-shot for special cases
if is_complex_query(query):
    prompt_with_examples = add_examples(query)
    response = model.query(prompt_with_examples)
else:
    response = model.query(query)
```

**3. Distillation + RAG**
- Distill for cost/speed
- RAG for factual grounding

```python
# Distilled GPT-4o-mini model
model = "zava-distilled-mini"

# Retrieve current data
context = retrieve_from_search(query)

# Query with context
response = model.query(f"Context: {context}\n\nQ: {query}")
```

---

## Training Data Best Practices

### 1. Quality Over Quantity

**Better:**
- 100개의 고품질의 다양한 예시
- 신중하게 선별 및 검증됨
- 실제 사용 사례를 대표함

**Worse:**
- 1000개의 저품질의 반복적인 예시
- 검토 없이 자동 생성됨
- 실제 쿼리를 대표하지 않음

### 2. Diversity in Training Data

Cover different:
- **Query types** (questions, requests, commands)
- **Complexity levels** (simple to multi-step)
- **Product categories** (paint, tools, hardware)
- **Customer intents** (search, compare, fact-check)

```jsonl
{"messages": [{"role": "user", "content": "What paint?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "user", "content": "Compare latex vs oil-based paint for outdoor furniture"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "user", "content": "Is PFIP000002 available in blue?"}, {"role": "assistant", "content": "..."}]}
```

### 3. Validation Split

**Don't use all data for training:**

```
Total: 1000 examples
  ↓
Training: 800 (80%)
Validation: 200 (20%)
```

**Use validation set to:**
- 과적합 감지
- 하이퍼파라미터 조정
- 일반화 성능 측정

### 4. Consistent Formatting

```jsonl
{"messages": [{"role": "system", "content": "You are Cora..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are Cora..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```

**Keep consistent:**
- 시스템 메시지 (모든 시스템에서 동일)
- 응답 구조
- 용어 및 명칭
- 어조 및 스타일

### 5. Token Budget Awareness

```python
# Check before training
def validate_training_data(data_file):
    total_tokens = 0
    
    with open(data_file) as f:
        for line in f:
            example = json.loads(line)
            tokens = count_tokens(example)
            total_tokens += tokens
            
            if tokens > 4096:  # Example too long
                print(f"Warning: Example exceeds limit: {tokens} tokens")
    
    epochs = 3
    cost = (total_tokens * epochs * 0.008) / 1000  # Example rate
    
    print(f"Total tokens: {total_tokens}")
    print(f"Estimated cost: ${cost:.2f}")
    
    return total_tokens < 1_000_000  # Example limit
```

### 6. Iterative Improvement

**Process:**
```
1. 초기 학습 데이터셋 생성 (100개 예제)
   ↓
2. Fine-tune model
   ↓
3. 검증 데이터셋에서 테스트
   ↓
4. 오류 패턴 파악
   ↓
5. 오류 해결을 위한 예제 추가
   ↓
6. 2~5단계 반복
```

**Each iteration improves specific weaknesses.**

---

## Evaluating Custom Models

### Before vs After Comparison

**Test on validation set:**

```python
validation_queries = [...]  # Held-out test set

# Test base model
base_results = evaluate_model("gpt-4o-mini", validation_queries)

# Test fine-tuned model
custom_results = evaluate_model("zava-fine-tuned", validation_queries)

# Compare
comparison = {
    "Base Model": {
        "Accuracy": base_results.accuracy,
        "Tone Match": base_results.tone_score,
        "Avg Tokens": base_results.avg_tokens
    },
    "Fine-Tuned": {
        "Accuracy": custom_results.accuracy,
        "Tone Match": custom_results.tone_score,
        "Avg Tokens": custom_results.avg_tokens
    }
}
```

### Key Metrics

**1. 작업 성능**
- 정확도가 향상되었습니까?
- 답변의 관련성이 높아졌습니까?
- 제품 추천이 더 나아졌습니까?

**2. 스타일 일관성**
- 브랜드 보이스와 일치합니까?
- 모든 문의에서 일관된 어조를 유지합니까?
- 적절한 격식 수준을 유지합니까?

**3. 효율성**
- 프롬프트 길이가 더 짧아졌습니까?
- 응답 속도가 빨라졌습니까?
- 토큰 사용량이 줄었습니까?

**4. 오류 감소**
- 잘못된 응답이 줄어들었습니까?
- 예외 상황을 더 잘 처리합니까?
- 오류 발생 시 더 자연스러운 응답을 제공합니까?

### A/B Testing in Production

```python
# Split traffic
def route_query(query):
    if random.random() < 0.5:
        return base_model.query(query)
    else:
        return fine_tuned_model.query(query)

# Track metrics
metrics = {
    "base_model": {"satisfaction": [], "latency": []},
    "fine_tuned": {"satisfaction": [], "latency": []}
}

# After 1000 queries, compare
analyze_ab_test(metrics)
```

### Regression Testing

**Ensure fine-tuning didn't break existing capabilities:**

```python
# Test suite
regression_tests = [
    {"input": "What is 2+2?", "expected": "4"},
    {"input": "What is the capital of France?", "expected": "Paris"},
    # ... general knowledge tests
]

# Both models should pass
assert all(test_model(base_model, regression_tests))
assert all(test_model(fine_tuned, regression_tests))
```

---

## Terminology Quick Reference

| Term | Simple Definition |
|------|-------------------|
| **Fine-Tuning** | 특정 데이터에 맞춰 모델을 추가로 학습시키는 과정 |
| **Distillation(증류)** | 큰 모델을 모방하도록 작은 모델을 학습시키는 과정 |
| **Few-Shot Prompting** | 프롬프트에 예시 응답을 포함하는 과정 |
| **Supervised Fine-Tuning (SFT)** | 레이블이 지정된 입력-출력 쌍을 사용하여 미세 조정하는 과정 |
| **JSONL** | JSON 라인 형식 - 한 줄에 하나의 JSON 객체 |
| **Training Data** | 모델에 패턴을 학습시키는 데 사용되는 예시 |
| **Validation Data** | 모델 성능 테스트를 위해 따로 보관해 둔 예시 |
| **Epoch** | 학습 데이터를 한 번 완전히 학습하는 과정 |
| **Token** | 청구에 사용되는 텍스트 단위(약 4자) |
| **Hyperparameters** | 학습을 제어하는 ​​설정(예: 학습률) |
| **Overfitting(과적합)** | 모델이 훈련 데이터를 암기하여 일반화하지 못함 |
| **Teacher Model** | 증류 과정에서 소스로 사용되는 대형 모델 |
| **Student Model** | 증류 과정에서 훈련되는 소형 모델 |
| **Custom Grader** | 응답 품질을 평가하는 함수 |

---

## What's Next?

이제 모델 맞춤 설정 개념을 이해했으니, 자신만의 모델을 세밀하게 조정하고 핵심 정보를 추출할 준비가 되었습니다!

### Hands-On Notebooks in This Section

- **`31-basic-finetuning.ipynb`** - Zava 제품 데이터를 사용하여 모델 미세 조정
  - JSONL 형식으로 학습 데이터 준비
  - 토큰 개수 검증 및 데이터 최적화
  - Azure OpenAI에 미세 조정 작업 제출
  - 미세 조정된 모델 배포 및 테스트
  - 기본 모델과 미세 조정된 모델의 성능 비교

- **`32-custom-grader.ipynb`** - 품질 관리를 위한 사용자 지정 평가기 구축
  - 사용자 지정 채점 함수 생성
  - 품질별로 학습 데이터 필터링
  - 학습 데이터 품질 향상
  - 응답이 표준을 충족하는지 검증

- **`33-distill-finetuning.ipynb`** - GPT-4o 지식을 GPT-4o-mini로 정제
  - GPT-4o에서 교사 응답 생성
  - 학생 학습 데이터 세트 생성
  - 교사 출력을 기반으로 GPT-4o-mini 미세 조정
  - 학생과 교사의 품질 비교
  - 비용 절감액 계산 증류

### Recommended Learning Path

1. **Start here** → 개념 이해 (this notebook)
2. **Next** → Basic fine-tuning (`31-basic-finetuning.ipynb`)
3. **Then** → Quality control (`32-custom-grader.ipynb`)
4. **Advanced** → Distillation (`33-distill-finetuning.ipynb`)
5. **After** → Move to evaluation labs (measure improvements)
6. **Finally** → 사용자 지정 모델을 프로덕션 환경에 배포

---

## Further Reading

For deeper understanding:

- **[Fine-Tuning Guide](https://learn.microsoft.com/azure/ai-services/openai/how-to/fine-tuning)** - Azure OpenAI fine-tuning documentation
- **[Preparing Training Data](https://learn.microsoft.com/azure/ai-services/openai/how-to/fine-tuning?tabs=python#prepare-training-data)** - Data format and best practices
- **[Model Distillation](https://learn.microsoft.com/azure/ai-studio/concepts/model-distillation)** - Knowledge transfer concepts
- **[Token Optimization](https://learn.microsoft.com/azure/ai-services/openai/how-to/token-optimization)** - Reducing costs
- **[Evaluation Metrics](https://learn.microsoft.com/azure/ai-studio/concepts/evaluation-metrics-built-in)** - Measuring model quality

---

Ready to fine-tune your first model? Open `31-basic-finetuning.ipynb` to get started! 🚀